# Project 2 : Inverted Pendulum with MPC

### Shanmukha Chaitanay Peddeti      ASU id: 1228632836

<img src="/1.jpg" width="" align="" />

<img src="/2.jpg" width="" align="" />

<img src="/3.jpg" width="" align="" />

<img src="/4.jpg" width="" align="" />

<img src="/5.jpg" width="" align="" />

In [1]:
!pip install cvxpy==1.4.1
!pip install cvxpylayers==0.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 101.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 24.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer
import numpy as np


n_state = 4
n_action = 1
T = 200
dt = 0.02
x_0 = np.array([0.,0.1,2.,0.])
total_time_step = 200

action_trajectory = []
state_trajectory = []

# Parameter
BOOST_ACCEL = 0.1  # thrust constant
FRAME_TIME = 0.01  # time interval
m = 1 # mass of the pendulum
M = 5 # mass of the cart
l = 0.6 # lenth of the pendulum
g = 9.81 # accleration due to gravity

def mpc(x_0, T):

    x = cp.Variable((n_state, T + 1))
    u = cp.Variable((n_action, T))
    theta0 = x_0[2]

    s = np.sin(x_0[2])
    c = np.cos(x_0[2])
    deno = M + m*(s**2)

    A = np.array([[1,dt, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, dt],
                  [0, 0, 0, 1]])

    B = np.array([[ 0],
                  [ BOOST_ACCEL*dt / deno],
                  [ 0],
                  [ BOOST_ACCEL*dt *(-s / (l*deno)) ],])

    c = np.array([0,(g*dt * m*s*c) / deno ,0, -(g*dt * (m+M)*s)/ (l*deno)])
    cost = 0
    constr = []
    for t in range(T):
        constr += [x[:, t + 1] == A @ x[:, t] + B @ u[:, t] + c, 
                   cp.abs(u[0, t]) <= 2, 
                #    cp.abs(u[1, t]) <= 2,
                   x[1, t] >= 0]
    # cost = cp.sum_squares(x[:, T])
    cost = 10 * cp.square(x[0, T]) + cp.sum_squares(x[:, T])

    # sums problem objectives and concatenates constraints.
    constr += [x[:, 0] == x_0]
    problem = cp.Problem(cp.Minimize(cost), constr)
    problem.solve()
    return x, u

def visualize(x,u):
    data = x
    action_data = u
    x = data[:, 0]
    vx = data[:, 1]
    theta = data[:, 2]%(2*torch.pi)
    vtheta = data[:, 3]
    force = action_data[:,0]
    frame = range(T)

    fig, ax = plt.subplots(2, 3, tight_layout = 1, figsize = (20, 10))

    ax[0,0].plot(frame, x, c = 'b', label = "X")
    ax[0,0].set_xlabel("Time Interval")
    ax[0,0].set_ylabel("x (m)")
    ax[0,0].set(title=f'Displacement of the cart at frame {self.epoch}')

    ax[0,1].plot(frame, vx, c = 'c', label = "Vx")
    ax[0,1].set_xlabel("Time Interval")
    ax[0,1].set_ylabel("Vx (m/s)")
    ax[0,1].legend(frameon=0)
    ax[0,1].set(title =f'Velocity of the cart at frame {self.epoch}')

    ax[1,0].plot(frame, theta, c = 'g', label = "Theta")
    ax[1,0].set_xlabel("Time Interval")
    ax[1,0].set_ylabel("Theta (rad)")
    ax[1,0].legend(frameon=0)
    ax[1,0].set(title=f'Angle of the pendulum at {self.epoch}')

    ax[1,1].plot(frame, vtheta, c = 'g', label = "V_theta")
    ax[1,1].set_xlabel("Time Interval")
    ax[1,1].set_ylabel("Angular velocity (rad/s)")
    ax[1,1].legend(frameon=0)
    ax[1,1].set(title=f'Angular velocity of the pendulum at {self.epoch}')

    ax[1,2].plot(frame, force, c = 'y', label = "Force")
    ax[1,2].set_xlabel("Time Interval")
    ax[1,2].set_ylabel("Force (N)")
    ax[1,2].legend(frameon=0)
    ax[1,2].set(title=f'Force on the cart at {self.epoch}')

    plt.show()

def simulate(state, action):
    s = torch.sin(state[0,2])
    c = torch.cos(state[0,2])
    deno = M + m*(s**2)

    # delta1 is second order PDE due to effect of dynamics
    delta1 = m*l*(state[0,3]**2)*(1/deno)*torch.tensor([[0., s, 0., -s*c*(1/l)]]) 
        
    # delta1 is second order PDE due to effect of gravity
    delta2 = g*(1/deno) * torch.tensor([0., m*s*c, 0., -(m+M)*s*(1/l)]) 
        
    # delta1 is second order PDE due to effect of action
    # delta3 = BOOST_ACCEL * (1/deno) * torch.mul(torch.tensor([[0., 1, 0., -s*(1/l)]]), action[0, 0].reshape(-1, 1))
    delta3 = (torch.mul(torch.tensor([[0., 1, 0., -s*(1/l)]]), action[0, 0].reshape(-1, 1)))/deno
        
    step_mat = torch.tensor([[1., FRAME_TIME, 0., 0.],
                                 [0., 1., 0., 0.],
                                 [0., 0., 1., FRAME_TIME],
                                 [0., 0., 0., 1.],])

    state = (step_mat @ state.T) + FRAME_TIME*(delta1.T + delta2.T + delta3.T) 
    state = state.T
    
    print(state)
    print(action)
    return state


def control(x_0, total_time_step):
    x_current = x_0
    
    for i in range(total_time_step):
        x, u = mpc(x_current, T-i)

        if u[:,0].value is None: # if the MPC problem is infeasible, stop
            print('MPC infeasible.')
            break
        
        # visualize(np.array(x[1:,:].value).T, np.array(u.value).T)
        action = u[:,0].value
        x_current = simulate(x_current, action)
        action_trajectory.append(action)
        state_trajectory.append(x_current)

control(x_0, total_time_step)


MPC infeasible.


In [3]:
!pip install ipywidgets
from ipywidgets import IntProgress
from IPython.display import display
from matplotlib import pyplot as plt, rc
from matplotlib.animation import FuncAnimation, PillowWriter
rc('animation', html='jshtml')


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
def animation(state_trajectory, action_trajectory):
        length = 0.10          # m
        width = 0.02          # m

        v_exhaust = 1     
        print("Generating Animation")
        steps = min(len(state_trajectory), len(action_trajectory))
        final_time_step = round(1/steps,2)
        f = IntProgress(min = 0, max = steps)
        display(f)

        data = np.array(state_trajectory)
        action_data = np.array(action_trajectory)

        x_t = data
        u_t = action_data
        print(x_t.shape, u_t.shape)

        fig = plt.figure(figsize = (5,10), constrained_layout=False)
        ax1 = fig.add_subplot(111)
        plt.axhline(y=-0.5, color='black', linestyle='--', lw=1)

        rod_fig, = ax1.plot([], [], linewidth = 4.5, color = 'black') # pendulum rod 
        pend_fig,= ax1.plot([], [], 'o', color = 'red', markersize = 12) # pendulum mass
        cart_fig,= ax1.plot([], [], 's', color = 'skyblue', markersize=20) # cart body
        force_fig,= ax1.plot([], [], linewidth = 6.5, color = 'green') # force line


        plt.tight_layout()

        ax1.set_xlim(-1,5 )
        ax1.set_ylim(-2, 3)
        ax1.set_aspect(1)  # aspect of the axis scaling, i.e. the ratio of y-unit to x-unit

        def update(i):

            x_1 = x_t[i,0]
            vx_1 = x_t[i,1]
            theta = x_t[i,2]
            vtheta_1 = x_t[i,3]

            cart_x = x_1 #+ vx_1 * FRAME_TIME
            pendulum_x =  cart_x + (l * math.sin(theta))
            pendulum_y = l * math.cos(theta)
        
            rod_fig.set_data([cart_x, pendulum_x], [0, pendulum_y])
            pend_fig.set_data(pendulum_x, pendulum_y)
            cart_fig.set_data(cart_x,0)

            force_1 = u_t[:i, 0]
            force_fig.set_data(cart_x+force_1 -0.02,0)

            f.value += 1

        playback_speed = 5000 # the higher the slower 
        anim = FuncAnimation(fig, update, np.arange(0, steps-1, 1), interval= final_time_step * playback_speed)

        # Save as GIF
        writer = PillowWriter(fps=20)
        anim.save("inverted_pendulum_mpc.gif", writer=writer)

animation(state_trajectory, action_trajectory)


Generating Animation


ZeroDivisionError: division by zero

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e7c2bc5a-8560-49b0-9204-5d9e2bfeb73a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>